## Installing the required packages


In [ ]:
%pip install datasets evaluate jiwer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
import datasets
import evaluate
import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import AutoModelForCTC
from transformers import TrainingArguments
from transformers import Trainer

In [ ]:
ds = datasets.load_dataset("SPRINGLab/asr-task-data") # English dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/356 [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/397M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/488M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/375M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/335M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

The function takes a batch of data as input. Each batch contains text samples. It concatenates all the text samples in the batch into a single string ` all_text `. It then creates a list of unique characters ` vocab ` from this combined text and returns a dictionary with two keys ` vocab ` containing the list of unique characters and ` all_text ` containing the combined text

In [ ]:
def extract_all_chars(batch):
    all_text=" ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab":[vocab], "all_text":[all_text]}

In [ ]:
vocabs = ds.map(extract_all_chars, batch_size=8)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

` vocab_list ` is initialized to store all the characters found across the dataset. We iterate over the vocabulary from each batch in the training set and extend ` vocab_list ` qith the characters from each batch's vocabulary.

In [ ]:
vocab_list = []

In [ ]:
for v in vocabs["train"]["vocab"]:
    vocab_list.extend(v[0])

In [ ]:
# remove duplicate characters
vocab_list = list(set(vocab_list))

In [ ]:
# map each character to integer id (starting from 0)

vocab_dict = {v: k for k, v in enumerate(vocab_list)}

In [ ]:
# reassign space character to pipe character, to make it more visible
vocab_dict["|"] = vocab_dict[" "]
del  vocab_dict[" "]

In [ ]:
# add unknown and pad token
# unknown token handles characters unseen during training that the model may encounter later
vocab_dict["[UNK]"]=len(vocab_dict)
vocab_dict["[PAD]"]=len(vocab_dict)
len(vocab_dict)

35

In [ ]:
import json
with open("vocab.json", "w") as f:
    json.dump(vocab_dict, f)

A ` Wav2Vec2CTCTokenizer ` is created using the saved ` vocab.json ` file.


It defines the following special tokens:

unk_token: `["UNK"]` for unknown characters

pad_token: ` ["PAD"] ` for padding


word_delimeter_token: ` "|" `, which is used as the token for separating words (since spaces were replaced by pipes earlier)

See the docs for ` Wav2Vec2CTCTokenizer `.

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", pad_token="[PAD]", unk_token="[UNK]", word_delimiter_token="|")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


A ` Wav2Vec2FeatureExtractor ` is initialized to process audio features

Key Parameters:

feature_size=1: indicates 1D audio features

sampling_rate=16000: Audio is expected to have a sampling rate of 16000 Hz

padding_value=0.0: Padding value is set to 0.0 (no signal/audio)

do_normalize=True: Normalization of audio signal is enabled (0 mean, 1 SD)

return_attention_mask=False: Attention mask are not used

See the docs for ` Wav2Vec2FeatureExtractor `.

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=False
                                             )

In [ ]:
# feature_extractor.to_json_file

` Wav2Vec2Processor ` combines feature_extractor and tokenizer.



In [ ]:
processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer
)

In [ ]:
print("Target text: ", ds["train"][10]["text"])
print("Input array shape:", np.array(ds["train"][10]["audio"]["array"]).shape)
print("Sampling rate:", ds["train"][10]["audio"]["sampling_rate"])

Target text:  so if you have it already downloaded
Input array shape: (41760,)
Sampling rate: 16000


The function prepares the data for training by extracting features

In [1]:
def prepare_dataset(batch):
    audio = batch["audio"]
    # batched output is unbatched to ensure mapping is correct
    batch["input_ids"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
ds = ds.map(prepare_dataset)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        # input
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            return_tensors="pt",
        )

        # labels
        with self.processor.as_target_processor():
            labels_batch=self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt"
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention.ne(1), -100)

        batch["labels"] = torch.tensor(labels, dtype=d_type)

        return batch

In [ ]:
data_collator=DataCollatorCTCWithPadding(processor=processor, padding=True)

We define an error metric and function to compute it while training

In [ ]:
wer_metric=evaluate.load("wer")

In [ ]:
def compute_metric(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids==-100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)

    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, referecnes=label_str)
    return {"wer":wer}

In [ ]:
ds = ds["train"].train_test_split(test_size=0.1, shuffle=True, seed=42)

Base pretrained model is defined and parameters are set for training.

Refer to Trainer and TrainingArguments

In [ ]:
model=AutoModelForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction = "mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_encoder() # freeze the feature encoder

In [ ]:

training_args = TrainingArguments(
    output_dir="AsrTaskModel",
    group_by_length=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=2,
    fp16=True,
    gradient_checkpointing=True,
    save_steps=1000,
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    save_strategy="steps"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metric,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=processor.feature_extractor
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


AttributeError: 'list' object has no attribute 'keys'